In [3]:
# Library imports

import pandas as pd
import numpy as np
import re

In [5]:
# read the source data file

df = pd.read_parquet('./data/source-data.parquet')

In [8]:
# review the dataframe

df.head()

,Client MAC Address,MAC Address,time,VLAN_ID,tonnage,activeTime,date,month,wday,geometry,type
0,00:00:00:57:b4:49,a8:9f:ec:fc:30:86,2019-11-22T17:00:00Z,822,0.245171,22.896,2019-11-22,Nov,Fri,"c(-75.160178, 39.942893)",outdoor
2,00:04:4b:25:23:3b,a8:9f:ec:da:ad:52,2019-11-11T02:00:00Z,1226,0.485443,125.977,2019-11-11,Nov,Mon,"c(-75.159325, 39.939919)",outdoor
1,00:04:4b:25:23:3b,00:0d:67:7b:aa:08,2019-11-23T05:00:00Z,843,0.002167,3.000,2019-11-23,Nov,Sat,"c(-75.157825, 39.944691)",outdoor
3,00:08:22:00:99:48,a8:9f:ec:da:b3:fa,2019-11-19T16:00:00Z,612,0.003054,3.724,2019-11-19,Nov,Tue,"c(-75.16045, 39.941096)",outdoor
5,00:08:22:00:99:48,a8:9f:ec:fc:30:b6,2019-11-19T16:00:00Z,1224,0.003543,8.515,2019-11-19,Nov,Tue,"c(-75.156092, 39.941646)",outdoor


In [7]:
# enabling sorted view of dataframe by client mac & time

df = df.sort_values(["Client MAC Address","time"], ascending=[True,True])

In [ ]:
# Since this dataframe contains lat, long in geometry form - we need to recapture the lat-long
# without getting into the complication of installing geopandas to do something that regex
# can also accomplish without exceeding complexity

# define regex expressions
lon_match = re.compile(r'c\((.*),\s+?.*\)')
lat_match = re.compile(r'c\(.*,\s+?(.*)\)')

# apply regex transformations
df['lat'] = df['geometry'].str.extract(lat_match)
df['lon'] = df['geometry'].str.extract(lon_match)